In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import datetime
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import gc
from collections import Counter
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
import re
from sklearn import preprocessing

In [75]:
feature_train=pd.read_pickle("./processed_data/df_train.pkl")
feature_test=pd.read_pickle("./processed_data/df_test.pkl")
user_train=pd.read_pickle("./processed_data/user_train.pkl")
user_test=pd.read_pickle("./processed_data/user_test.pkl")
feature_test["label"]=0

In [76]:
feature_train.dev_height[feature_train.dev_height==0]=None
feature_train.dev_width[feature_train.dev_width==0]=None
feature_train.dev_ppi[feature_train.dev_ppi==0]=None
feature_test.dev_height[feature_test.dev_height==0]=None
feature_test.dev_width[feature_test.dev_width==0]=None
feature_test.dev_ppi[feature_test.dev_ppi==0]=None

In [78]:
feature_train=pd.merge(feature_train,user_train,left_on="android_id",right_on="uid",how="left",right_index=True)
feature_test=pd.merge(feature_test,user_test,left_on="android_id",right_on="uid",how="left",right_index=True)

In [154]:
feature_train.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'label', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'Day', 'Hour', 'minute', 'apptype_most_index', 'apptype_most_freq',
       'carrier_most_index', 'carrier_most_freq', 'lan_most_index',
       'lan_most_freq', 'media_id_most_index', 'media_id_most_freq',
       'ntt_most_index', 'ntt_most_freq', 'os_most_index', 'os_most_freq',
       'osv_most_index', 'osv_most_freq', 'package_most_index',
       'package_most_freq', 'version_most_index', 'version_most_freq',
       'location_most_index', 'location_most_freq', 'fea1_hash_most_index',
       'fea1_hash_most_freq', 'cus_type_most_index', 'cus_type_most_freq',
       'Day_most_index', 'Day_most_freq', 'Hour_most_index', 'Hour_most_freq',
       'dev_height_min', 'dev_height_max', 'dev_height_mean',
       'dev_height_median', 'dev_height_nunique', 'dev_ppi_

In [207]:
cate_feature = ['apptype', 'carrier', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'location', 'fea1_hash', 'cus_type']
# for item in cate_feature:
#     le=preprocessing.LabelEncoder()
#     feature_train[item] = le.fit_transform(feature_train[item])
#     feature_test[item] = le.transform(feature_test[item])

prediction = feature_test[['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']]
# prediction = feature_test[['apptype', 'carrier', 'dev_height', 'dev_ppi','timestamp',
#        'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
#        'Day', 'Hour']]


prediction['label'] = 0


y_col = 'label'
x_col=['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']
# x_col = ['apptype', 'carrier', 'dev_height', 'dev_ppi','timestamp',
#        'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
#        'Day', 'Hour', 'apptype_most_index', 'apptype_most_freq', 'carrier_most_index', 'carrier_most_freq',
#        'lan_most_index', 'lan_most_freq', 'media_id_most_index',
#        'media_id_most_freq', 'ntt_most_index', 'ntt_most_freq',
#        'os_most_index', 'os_most_freq', 'osv_most_index', 'osv_most_freq',
#        'package_most_index', 'package_most_freq', 'version_most_index',
#        'version_most_freq', 'location_most_index', 'location_most_freq',
#        'fea1_hash_most_index', 'fea1_hash_most_freq', 'cus_type_most_index',
#        'cus_type_most_freq', 'Day_most_index', 'Day_most_freq',
#        'Hour_most_index', 'Hour_most_freq']

model = lgb.LGBMClassifier(
    boosting_type='gbdt',
                           num_leaves=100,
                           max_depth=7,
                           learning_rate=0.03,
                           n_estimators=10000,
                           subsample=0.7,
                           feature_fraction=0.7,
                           reg_alpha=0.3,
                           reg_lambda=0.3,
                           random_state=220,
                           metric=None,
                           n_jobs=-1,
                           save_binary=True,
                        #    max_bin=255
                        #    cat_smooth=180
                        #    colsample_bytree=0.8
                        #    metric=["binary_error"]
                        #    max_bin=10,
                        #    device="gpu"
                           )

In [208]:
oof = []
df_importance_list = []
n=5
kfold = KFold(n_splits=n,shuffle=True,random_state=20)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(feature_train[x_col], feature_train[y_col])):
    X_train = feature_train.iloc[trn_idx][x_col]
    Y_train = feature_train.iloc[trn_idx][y_col]

    X_val = feature_train.iloc[val_idx][x_col]
    Y_val = feature_train.iloc[val_idx][y_col]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(
        X_train,
        Y_train,
        categorical_feature=cate_feature,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, Y_train), (X_val, Y_val)],
        verbose=100,
        eval_metric=["binary_logloss","binary_error"],
        early_stopping_rounds=300
    )

    pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = feature_train.iloc[val_idx].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(feature_test[x_col], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['label'] += pred_test / n

    df_importance = pd.DataFrame({
        'column': x_col,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)


Fold_1 Training ================================

Training until validation scores don't improve for 300 rounds
[100]	train's binary_logloss: 0.32049	train's binary_error: 0.120235	valid's binary_logloss: 0.325345	valid's binary_error: 0.12256
[200]	train's binary_logloss: 0.297412	train's binary_error: 0.1149	valid's binary_logloss: 0.310079	valid's binary_error: 0.11887
[300]	train's binary_logloss: 0.287222	train's binary_error: 0.111507	valid's binary_logloss: 0.305738	valid's binary_error: 0.11719
[400]	train's binary_logloss: 0.278574	train's binary_error: 0.108732	valid's binary_logloss: 0.30311	valid's binary_error: 0.11581
[500]	train's binary_logloss: 0.272385	train's binary_error: 0.106313	valid's binary_logloss: 0.302282	valid's binary_error: 0.11504
[600]	train's binary_logloss: 0.266231	train's binary_error: 0.103935	valid's binary_logloss: 0.301495	valid's binary_error: 0.11476
[700]	train's binary_logloss: 0.260557	train's binary_error: 0.101708	valid's binary_logloss:

In [209]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,fea_hash,13685.6
1,dev_height,11820.2
2,location,9206.4
3,cus_type,8558.8
4,fea1_hash,6276.0
5,dev_width,6043.4
6,dev_ppi,3457.2
7,package,2597.0
8,media_id,2470.2
9,apptype,1955.6


In [223]:
a = pd.DataFrame(prediction.index)
a['label'] = lgb_model.predict(prediction[x_col])

In [110]:
# a.to_csv('submission_0605_0124.csv', index=False)

In [224]:
user_label=pd.DataFrame()
user_label["uid"]=feature_train.groupby("android_id").count().index
user_label=user_label.set_index("uid")

In [225]:
user_label["label_prior"]=feature_train.groupby("android_id").label.mean()

In [226]:
a["uid"]=feature_test.android_id.values

In [227]:
a=pd.merge(a,user_label,on="uid",how="left")

In [228]:
def post(label,prior):
    n=len(label)
    count=0
    for i in range(n):
        if 0<=prior[i]<=0.3 and label[i]==1:
            label[i]=0
            count+=1
            # print(i)
        elif 0.7<=prior[i]<=1 and label[i]==0:
            label[i]=1
            count+=1
            # print(i)
        else:
            pass
    print(count)
    return label.values

In [229]:
a.label=post(a.label,a.label_prior)

406


In [230]:
a=a[["sid","label"]]
a.to_csv('submission_0605_0226.csv', index=False)

In [231]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1


## 实验

In [403]:
# a=feature_train.groupby(["android_id","package"])["label"].agg(["mean","count"])
a=feature_train.groupby(["android_id","media_id"])["label"].mean()

In [404]:
a

android_id  media_id
0           0           0.103597
            3           0.285714
            6           0.845617
            7           0.000000
            8           0.029891
                          ...   
709888      20          1.000000
709891      7           0.000000
709892      0           1.000000
709895      35          1.000000
709898      60          0.000000
Name: label, Length: 362649, dtype: float64

In [405]:
def fun1(x):
    if x>=0.5:
        return 1
    else:
        return 0

In [406]:
c=pd.merge(feature_train,a,on=["android_id","media_id"],how="left")

In [407]:
c.label_y=c.label_y.apply(fun1)

In [408]:
count=0
d=np.array(c.label_y)
e=np.array(feature_train.label)
for i in range(len(c.label_y)):
    if d[i]==e[i]:
        count+=1
print(count/500000)

0.96905


In [409]:
gc.collect()

1733

In [258]:
feature_train[['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']][feature_train.label==0].head(20)

,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,version,fea_hash,location,fea1_hash,cus_type
sid,,,,,,,,,,,,,,,,
1093419,49,2,2214.0,NaN,1080.0,7,25,1,1,8,1,1,3562553457,4,152,24
1387136,68,2,NaN,NaN,NaN,7,8,0,1,9,99,1,369486452,7,480,6
1910989,31,1,NaN,NaN,NaN,7,4,5,1,8,1,8,3307888762,9,364,31
1842043,12,4,1920.0,NaN,1080.0,7,11,5,1,5,10,1,2675936369,11,121,2
1806212,29,2,2130.0,3.0,1080.0,9,0,5,0,9,137,14,3213977823,13,163,29
1671763,49,2,1920.0,480.0,1080.0,8,26,5,1,7,24,12,3180341175,14,99,29
1948217,83,2,960.0,NaN,540.0,7,7,1,1,5,6,1,1854118439,15,275,8
1212520,36,2,2280.0,NaN,1080.0,9,3,5,0,8,3,7,738486495,13,336,1
1311419,48,2,NaN,480.0,NaN,9,54,1,0,8,68,2,3055065311,16,416,5


In [259]:
feature_train[['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type']][feature_train.label==1].head(20)

,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,version,fea_hash,location,fea1_hash,cus_type
sid,,,,,,,,,,,,,,,,
1438873,81,2,NaN,NaN,NaN,7,20,5,1,9,18,13,2135019403,1,301,16
1185582,31,1,NaN,NaN,NaN,7,4,5,1,8,1,8,2782306428,2,353,37
1555716,22,1,760.0,NaN,360.0,7,0,0,1,8,1,1,1392806005,3,73,21
1400089,62,2,2280.0,NaN,1080.0,9,13,1,1,8,1,9,2364522023,5,189,15
1253737,18,2,2196.0,480.0,1080.0,9,35,5,0,8,34,9,1655793887,6,126,7
1597216,48,2,760.0,NaN,360.0,9,6,1,0,8,5,1,3912031003,8,326,6
1785622,48,2,760.0,NaN,360.0,9,6,1,0,8,5,1,1485855711,10,336,6
1935139,48,2,851.0,NaN,393.0,9,6,1,0,9,5,1,2979808474,12,326,6
1379250,40,0,NaN,NaN,NaN,7,15,5,1,7,16,1,3369125340,17,15,0
